## Implementação do Algorítmo KNN com funções básicas do Python
##### Implementado por: Augusto Modesto

#### Algumas observações quanto ao exercício:

##### 1- Foram utilizadas apenas funções básicas do python para implementação do algorítmo, com exceção de algumas bibliotecas específicas.
##### 2- Foram utilizados métodos atômicos nas classes criadas para facilitar possíveis testes unitários.
##### 3- Foram criadas duas classes para realizar o exercício.
##### 4- Foram criados dois casos de testes para testar o código do algorítmo criado
##### 5- Foram comparados os dois casos de testes do modelo criado com outros dois casos de testes utilizando a biblioteca sklearn (final do notebook)

### Bibliotecas essenciais para construção do algorítmo

In [1295]:
import math # Utilizada para calcular a raiz quadrada
import csv # Utilizada para abrir o arquivo CSV e carregar na memória
import random # Utilizada para gerar uma aleatoriedade para separar as bases de treino e teste

### Classes para execução do exercício

#### Classe para manipulação de arquivos e dados:

In [1296]:
class ManipulatesFileData:
    '''
    Esta classe é responsável por fornecer métodos que permitam a manipulação de arquivos e dados para
    trabalhar, especificamente, com algorítmos de Machine Learning (ML).
    
    Lista de Atributos:
        N/A
    
    Lista de Métodos Públicos:
        load_dataset = carrega um dataset na memoria.
        train_test_split = separa um dataset em duas listas (variáveis repostas e variáveis preditoras) de treino
                           e duas listas (variáveis repostas e variáveis preditoras) de teste, utilizando o método
                           de validação cruzada holdout.
        detail_datasets_train_test = apresenta na tela um detalhamento das listas de treino e teste
    
    Lista de Métodos Privados:
        __shuffle_dataset = randomiza um dataset.
        __separate_dataset = separa o dataset em duas listas de treino e teste.
        __remove_predictor_element = remove o elemento preditor das lista de treino e teste.
    '''
    
    def load_dataset(self, path_file):
        '''
        Método público que carrega um csv na memória do nootbook.
        Retorna a base de dados em formato de lista.
        
        Args:
            path_file(str): O caminho do arquivo CSV. Caso esteja na mesma pasta do nootbook, basta inserir o
                            nome do arquivo. Não é necessário adicionar a extensão do arquivo (.csv).
        
        Returns:
            dataset(list): Uma lista com os dados do CSV de entrada carregado na memória.
        '''
        with open(path_file) as csv_file:
            dataset = list(csv.reader(csv_file))
        return dataset

    def __shuffle_dataset(self, dataset, test_ratio):
        '''
        Método privado que embaralha o dataset, isto é, randomiza a base de dados.
        Retorna a base de dados randomizada e a proporção da base para teste do modelo.
        
        Args:
            dataset(list): A base de dados contendo os dados em formato de lista carregados na memória.
            
            test_ratio(float): Tamanho (proporção) da base de teste em valor percentual.
            
        Returns:
            shuffled_dataset(list): Uma lista com os dados do dataset randomizados (embaralhados).
            
            test_size(int): Tamanho (inteiro) da base de dados em valor absoluto (Ex: 200 linhas do dataset randomizado). 
        '''
        shuffled_dataset = random.sample(dataset, len(dataset))
        test_size = int(len(dataset) * test_ratio)        
        return shuffled_dataset, test_size 
    
    def __separate_dataset(self, shuffled_dataset, test_size):
        '''
        Método privado que separa um dataset, preferencialmente randomizado (embaralhados), em duas lista: uma para treino
        e outra para teste.
        Retorna as duas listas de treino e teste.
        
        Args:
            shuffled_dataset(list): Uma lista com os dados do dataset, preferencialmente randomizados (embaralhados).
            
            test_size(float): Tamanho (proporção) da base de teste.
            
        Returns:
            train_data(list): Uma lista com os dados para treinamento do modelo, de acordo com a proporção
            fornecida de teste.
            
            test_data(list): Uma lista com os dados para teste do modelo, de acordo com a proporção
            fornecida de teste.
        '''
        test_data = shuffled_dataset[:test_size]
        train_data = shuffled_dataset[test_size:]
        return train_data, test_data
    
    def __remove_predictor_element(self, train_data, test_data):
        '''
        Método privado que remove o elemento preditor, considerando como sendo o último elemento, do dataset.
        Retorna quatro lista contendo os elementos seletores (varíaveis respostas) e os valores preditores
        de treino e teste.
        
        Args:
            train_data(list): Uma lista com os dados para treinamento do modelo, de acordo com a proporção
            fornecida de teste.
            
            test_data(list): Uma lista com os dados para teste do modelo, de acordo com a proporção
            fornecida de teste.
        
        Returns:
            train_x(list): Uma lista contendo os elementos seletores (varíaveis respostas) da lista de treino.
            
            train_y(list): Uma lista contendo os elementos preditores (varíaveis preditoras) da lista de treino.
            
            test_x(list): Uma lista contendo os elementos seletores (varíaveis respostas) da lista de teste.
            
            test_y(list): Uma lista contendo os elementos preditores (varíaveis preditoras) da lista de teste.
        '''
        last_element = -1
        train_x = [sample[:last_element] for sample in train_data]
        train_y = [sample[last_element] for sample in train_data]
        test_x = [sample[:last_element] for sample in test_data]
        test_y = [sample[last_element] for sample in test_data]
        return train_x, train_y, test_x, test_y
    
    def train_test_split(self, dataset, test_ratio=0.3):
        '''
        Método público que separa uma base de dados em quatro listas para treino e teste de um modelo de machine learning
        utilizando o método de validação cruzada holdout.
        Retorna quatro lista contendo os elementos seletores (varíaveis respostas) e os valores preditores
        de treino e teste.
        
        Args:
            dataset(list): A base de dados contendo os dados em formato de lista carregados na memória.
            
            test_ratio(float): Tamanho (proporção) da base de teste em valor percentual. Por default o valor
            desta variável é 0.3 (30%).
        
        Returns:
            train_x(list): Uma lista contendo os elementos seletores (varíaveis respostas) da lista de treino.
            
            train_y(list): Uma lista contendo os elementos preditores (varíaveis preditoras) da lista de treino.
            
            test_x(list): Uma lista contendo os elementos seletores (varíaveis respostas) da lista de teste.
            
            test_y(list): Uma lista contendo os elementos preditores (varíaveis preditoras) da lista de teste.
        '''
        shuffled_dataset, test_size = self.__shuffle_dataset(dataset, test_ratio)
        train_data, test_data = self.__separate_dataset(shuffled_dataset, test_size)
        train_x, train_y, test_x, test_y = self.__remove_predictor_element(train_data, test_data)
        return train_x, train_y, test_x, test_y
    
    def detail_datasets_train_test(self, train_x, train_y, test_x, test_y):
        '''
        Método público que apresenta na tela um breve detalhamento das listas de treino (train) e teste (test)
        Não há retorno para este método.
        
        Args:
            train_x(list): Uma lista contendo os elementos seletores (varíaveis respostas) da lista de treino.
            
            train_y(list): Uma lista contendo os elementos preditores (varíaveis preditoras) da lista de treino.
            
            test_x(list): Uma lista contendo os elementos seletores (varíaveis respostas) da lista de teste.
            
            test_y(list): Uma lista contendo os elementos preditores (varíaveis preditoras) da lista de teste.
        
        Returns:
            N/A
        '''
        print('Tamanho de train_x:', len(train_x), 'Tamanho de train_y:', len(train_y),
              '\nTamanho de test_x:', len(test_x), 'Tamanho de test_y:', len(test_y))

        print('\nPrimeiros valores de train_x:\n', train_x[:3],
              '\nPrimeiros valores de train_y:\n', train_y[:3],
              '\nPrimeiros valores de test_x:\n', test_x[:3],
              '\nPrimeiros valores de test_y:\n', test_y[:3])

#### Classe contendo os principais métodos do algorítmo KNN:

In [1340]:
class KNeighborsClassifier:
    '''
    Esta classe contém os métodos para utilização do classificador KNN (k-nearest neighbors). O nome
    'KNeighborsClassifier' desta classe foi utilizado para manter o mesmo padrão do classificador original
    (classe KNeighborsClassifier).
    
    Lista de Atributos:
        k(int) = é o valor de k do classificador KNN. Neste classificador, K representa o k-ésimo vizinho mais
                 próximo das amostras da base de treino.

        distance_calculation_method(str) = é o método de cálculo da distância entre os pontos do plano (amostras
                                           treino).
        
        x_train(list) = é a lista de pontos utilizada para o 'treino' do algorítmo. Embora, o classificador KNN
                        não realize o treino das instâncias de teste, mas é necessário armazenar os pontos de 
                        treino na memória do programa para que seja possível calcular a distância entre os pontos
                        de treino e teste.
        
        y_train(list) = é a lista de pontos contendo os resultados de treino. Podem assumir os tipos str, int
                        ou float.
    
    Lista de Métodos Públicos:
        __init__ = inicializa as variáveis k e distance_calculation_method.
        fit = treina o modelo com base no dataset de entrada.
        predict = prediz os valores do dataset de teste com base no treino realizado com a função .fit().
        prediction_report = apresenta um relatório dos resultados da predição.
        accuracy_report = apresenta um relatório da acurácia da predição.
        confusion_matrix = apresenta uma matriz de confução da predição.
        
    
    Lista de Métodos Privados:
        __calculate_distance = cálcula a distância de dois pontos (p e q).
        __append_distance = realiza um append numa lista dos resultados do cálculo de distância.
        __find_shorter_distance = encontra as menores distância numa lista de acordo com o valor da variável k.
        __find_label = encontra as labels da lista de entrada.
        __predict_label = realiza a predição da label conforme lista de entrada.
        __discretize_predictors = discretiza os valores das listas de entrada.
    
    '''

    def __init__(self, k=3, distance_calculation_method='euclidean'):
        '''
        Método público utilizado para inicializar as principais variáveis do classificador.
        Não há retorno para este método.
        
        Args:
            k(int): é o valor de k do classificador KNN. Neste classificador, K representa o k-ésimo vizinho mais
                    próximo das amostras da base de treino. Por default o valor de k é 3.

            distance_calculation_method(str): é o método de cálculo da distância entre os pontos do plano (amostras
                                              treino). Por default o valor de distance_calculation_method é
                                              'euclidean'.
        
        Returns:
            N/A
        '''
        self.k = k
        self.distance_calculation_method = distance_calculation_method
    
    def fit(self, x_train, y_train):
        '''
        Método público utilizado para setar (inicializar) a base de dados de train (variáveis resultados e
        preditoras). Embora, o classificador KNN não realize o treino das instâncias de teste, mas é necessário
        armazenar os pontos de treino na memória do programa para que seja possível calcular a distância entre
        os pontos de treino e teste. O nome 'fit' foi utilizado para manter o mesmo padrão do classificador
        original (classe KNeighborsClassifier).
        Não há retorno para este método.
        
        Args:
            x_train(list): é a lista de pontos utilizada para o 'treino' do algorítmo. Embora, o classificador KNN
                           não realize o treino das instâncias de teste, mas é necessário armazenar os pontos de 
                           treino na memória do programa para que seja possível calcular a distância entre os
                           pontos de treino e teste.
        
            y_train(list): é a lista de pontos contendo os resultados de treino. Podem assumir os tipos str,
                           int ou float.
        
        Returns:
            N/A
        '''
        self.x_train = x_train
        self.y_train = y_train
    
    def __calculate_distance(self, point_p, point_q):
        '''
        Método privado utilizado para cálcular a distância entre dois pontos (point_p e point_q). É possível
        calcular a distância utilizando dois métodos distintos: a distância euclidiana; e a distância de manhattan.
        Para isso, se deve setar corretamente a variável distance_calculation_method.
        A distância euclidiana é dada por: raiz quadrada do somatório da diferença entre os pontos p e q elevados
        ao quadrado. Fórmula: sqrt(sum_points), onde sum_points += (float(point_p[i]) - float(point_q[i]))**2.
        A distância de manhattan é dada por: somatório do módulo da diferença entre os pontos p e q.
        Fórmula: sum_points += abs(float(point_p[i]) - float(point_q[i])).
        Retorna o resultado do cálculo da distância de dois pontos (p e q).
        
        Args:
            point_p(list): é a lista de pontos (variáveis resultados) do i-ésimo elemento de treino, onde i
                            corresponde a um sample.
        
            point_q(list): é a lista de pontos (variáveis resultados) do i-ésimo elemento de teste, onde i
                            corresponde a um sample. Essa varíavel corresponde a instância de teste corrente
                            (a instância de teste a ser calculada sua distância em relação aos i-ésimos elementos
                            de treino).
        
        Returns:
            calculation_result(float): é o resultado (float) do cálculo da distância dos pontos p e q. 
        '''
        sum_points = 0
        calculation_result = 0
        dimension_point = len(point_p)
        
        # Cálculo da distância euclidiana. Fonte: https://pt.wikipedia.org/wiki/Dist%C3%A2ncia_euclidiana
        if self.distance_calculation_method == 'euclidean': 
            for i in range(dimension_point):
                sum_points += (float(point_p[i]) - float(point_q[i]))**2
            calculation_result = math.sqrt(sum_points)
        
        # Cálculo da distância de manhattam. Fonte: https://pt.wikipedia.org/wiki/Geometria_pombalina
        elif self.distance_calculation_method == 'manhattan':
            for i in range(dimension_point):
                sum_points += abs(float(point_p[i]) - float(point_q[i]))
            calculation_result = sum_points
        
        return calculation_result
    
    def __append_distance(self, instance_test):
        '''
        Método privado que realizar um append (adiciona um elemento a lista) nos resultados do cálculo da 
        distância dos pontos p e q de cada sample.
        Retorna uma lista contendo todos os resultados do cálculo da distância de dois pontos (p e q) de cada
        sample.
        
        Args:
            instance_test(list): é um sample da lista de teste. Contém os valores das varíaveis resultados de
            uma instância de teste (sample).
            
        Returns:
            append_result(list): é uma lista contendo todos os resultados do cálculo da distância de dois
            pontos (p e q) de cada sample de entrada.
        '''
        append_result = []
        dimension_x_train = len(self.x_train)
        for sample in range(dimension_x_train):
            append_result.append(self.__calculate_distance(self.x_train[sample], instance_test))
        return append_result
    
    def __find_shorter_distance(self, distance_result):
        '''
        Método privado que encontra a menor distância da lista de resultado de cálculo da distâncias entre dois
        pontos (p e q).
        Retorna os k elementos de menor valor do vetor de entrada.
        
        Args:
            distance_result(list): é uma lista contendo todos os resultados do cálculo da distância entre dois
            pontos (p e q).
        
        Returns:
            k_shorter_distances(list): é uma lista com os k menores distâncias da lista de entrada.      
        '''
        ranged_distance_result = range(len(distance_result))
        k_shorter_distances = sorted(ranged_distance_result, key=lambda i: distance_result[i])[:self.k]
        return k_shorter_distances
    
    def __find_label(self, k_shorter_distances):
        '''
        Método privado que encontra as labels da lista de k menores distâncias.
        Retorna uma lista com os labels dos k menores distâncias da lista de entrada.
        
        Args:
            k_shorter_distances(list): é uma lista com os k menores distâncias.
        
        Returns:
            k_labels(list): é uma lista com os labels dos k menores distâncias da lista de entrada.
        '''
        k_labels = []
        for i in k_shorter_distances:
            k_labels.append(self.y_train[i])
        return k_labels
    
    def __predict_label(self, labels_k):
        '''
        Método privado que cálcula, utilizando a função max(..., key=labels_k.count), a quantidade de labels
        mais frequente na lista de labels de entrada.
        Retorna a label que apereceu com maior frequência na lista de entrada.
        
        Args:
            labels_k(list): é uma lista com os labels dos k menores distâncias.
            
        Returns:
            max_label(str): é a label que aparece com maior frequência na lista de entrada.
        '''
        max_label = max(labels_k, key=labels_k.count)
        return max_label

    def predict(self, x_test):
        '''
        Método público que realiza a predição da lista de entrada com base no dataset de treino inicializado
        com o método fit(). O nome 'predict' foi utilizado para manter o mesmo padrão do classificador original
        (classe KNeighborsClassifier).
        Retorna uma lista do resultado da predição.
        
        Args:
            x_test(list): é uma lista com as variáveis resultados (x_test) do dataset de teste.
            
        Returns:
            predict_result(list): é uma lista com os resultados da predição da lista de teste de entrada.
        '''
        predict_result = []
        for instance_x_test in x_test:
            distance_result = self.__append_distance(instance_x_test)
            k_shorter_distances = self.__find_shorter_distance(distance_result)
            k_labels = self.__find_label(k_shorter_distances)
            max_label = self.__predict_label(k_labels)
            predict_result.append(max_label)                
        return predict_result
    
    def prediction_report(self, predictions, y_test):
        '''
        Método público que apresenta na tela o resultado da predição de cada sample. Seguinte o padrão:
        Teste Nº: [sample]        Real: [valor real]        Predito: [valor predito]
        Não há retorno para este método.
        
        Args:
            predict_result(list): é uma lista com os resultados da predição.
        
            y_test(list): é a lista de pontos contendo os resultados de teste. Podem assumir os tipos str,
                          int ou float.
        
        Returns:
            N/A
        '''
        for sample, prediction in enumerate(predictions):
            print('Teste Nº:', (sample+1) , '\t Real:', y_test[sample], '\t Predito:', prediction)

    def accuracy_report(self, predictions, y_test):
        '''
        Método público que apresenta na tela a acurácia do resultado da predição de cada sample. Seguinte
        o padrão:
        Quantidade de Treinos: [tamanho do dataset de treino]
        Quantidade de Testes: [tamanho do dataset de teste]
        Quantidade de Acertos: [número de acertos quando o valor do dataset de teste é igual ao valor predito]
        Quantidade de Erros: [diferença do tamanho dataset de teste pela quantidade de acertos]
        Acurácia do Modelo: [Percentual de acertos em relação a quantidade de elemento no dataset de teste]
        Não há retorno para este método.

        Args:
            predict_result(list): é uma lista com os resultados da predição.
        
            y_test(list): é a lista de pontos contendo os resultados de teste. Podem assumir os tipos str,
                          int ou float.
        
        Returns:
            N/A
        '''
        hits = 0
        for prediction, sample in zip(predictions, y_test):
            if prediction == sample:
                hits += 1
                
        print('Quantidade de Treinos:', len(self.x_train))
        print('Quantidade de Testes:', len(y_test))
        print('Quantidade de Acertos:', hits)
        print('Quantidade de Erros:', (len(y_test)-hits))
        print('Acurácia do Modelo: %.2f%%' % (hits*100/len(y_test)))
    
    def __discretize_predictors(self, predictions, y_test):
        '''
        Método privado que transforma as labels preditivas em valores discretos.
        Retorna a quantidade de classes existentes nas labels preditivas e os datasets de preditores e teste
        discretizados.
        
        Args:
            predict_result(list): é uma lista com os resultados da predição.
        
            y_test(list): é a lista de pontos contendo os resultados de teste. Podem assumir os tipos str,
                          int ou float.
                          
        Returns:
            quantity_classes(inst): é a quantidade de classes presentes na lista de valores preditos. 
            
            predictors_discrete_list(list): é a lista dos valores preditos discretizados.
            
            test_discrete_list(list): é a lista de pontos contendo os resultados de teste discretizados.
        '''
        predictors_discrete_list = []
        test_discrete_list = []
        class_list = []
        
        # Verificando as classes existentes em predictions e populando em uma nova lista de classes (class_list)
        for sample in predictions:
            if sample not in class_list:
                class_list.append(sample)
        
        # Ordenando os valores presentes na lista de classes e contabilizando a quantidade de classes dessa lista
        class_list.sort()        
        quantity_classes = len(class_list)
        
        for prediction in predictions:            
            predictors_discrete_list.append(class_list.index(prediction))
        
        for sample in y_test:
            test_discrete_list.append(class_list.index(sample))        
        
        return quantity_classes, predictors_discrete_list, test_discrete_list

                
    def confusion_matrix(self, predictions, y_test):
        '''
        Método público que apresenta na tela uma matriz de confução dos valores preditos.
        Não há retorno para este método.
        
        Args:
            predict_result(list): é uma lista com os resultados da predição.
        
            y_test(list): é a lista de pontos contendo os resultados de teste. Podem assumir os tipos str,
                          int ou float.
        
        Returns:
            N/A
        '''
        quantity_classes, predictors_discrete_list, test_discrete_list = self.__discretize_predictors(predictions, y_test)
        confusion_matrix = []
        center_class = int(quantity_classes/2)
        
        # Cria uma matriz de confusão de tamanho [NxN] onde N é a quantidade de classes preditoras
        for line in range(quantity_classes):
            confusion_matrix.append(list())
            for aux in range(quantity_classes): 
                confusion_matrix[line].append(0)
        
        # Alimenta a matriz de confusão
        for prediction, sample in zip(predictors_discrete_list, test_discrete_list):
            if prediction == sample:
                confusion_matrix[sample][prediction] += 1
            else:
                confusion_matrix[sample][prediction] += 1
        
        # Imprime na tela a matriz de confusão em formato de tabela
        print('\t\tValores Preditos')
        print('-----------------------------------')
        for current_class in range(quantity_classes):
            if current_class == center_class:
                print('Valores Reais\t|', confusion_matrix[current_class], '\n')
            else:
                print('\t\t|', confusion_matrix[current_class], '\n')

### Realização dos testes das classes criadas

#### Carregando a base de dados na memória do programa e criando as listas de treino e teste:
##### Utilizando 20% do dataset original como teste (predição) e 80% para treino.

In [1389]:
# Instanciando um objeto para inicializar o dataset
MFD = ManipulatesFileData()

# Carregando o dataset na memória do programa
dataset = MFD.load_dataset('iris.data')

# Separando as listas de treino e teste
train_x, train_y, test_x, test_y = MFD.train_test_split(dataset, test_ratio=0.2)

# Apresentando um detalhamento das listas de treino e teste
MFD.detail_datasets_train_test(train_x, train_y, test_x, test_y)

Tamanho de train_x: 120 Tamanho de train_y: 120 
Tamanho de test_x: 30 Tamanho de test_y: 30

Primeiros valores de train_x:
 [['6.1', '3.0', '4.6', '1.4'], ['6.0', '2.9', '4.5', '1.5'], ['4.6', '3.1', '1.5', '0.2']] 
Primeiros valores de train_y:
 ['Iris-versicolor', 'Iris-versicolor', 'Iris-setosa'] 
Primeiros valores de test_x:
 [['6.3', '3.3', '6.0', '2.5'], ['5.9', '3.0', '5.1', '1.8'], ['5.1', '3.4', '1.5', '0.2']] 
Primeiros valores de test_y:
 ['Iris-virginica', 'Iris-virginica', 'Iris-setosa']


#### Caso de Teste 1: Utilizando o método euclidiano para cálculo da distância e k impar (5):

In [1390]:
# Instanciando o modelo KNN a partir do código criado
KNN = KNeighborsClassifier(k=9, distance_calculation_method='euclidean')

# Treinando o modelo
KNN.fit(train_x, train_y)

In [1391]:
# Realizando as predições com base no modelo treinado
prediction_result = KNN.predict(test_x)

In [1392]:
# Visualizando os testes realizados
KNN.prediction_report(prediction_result, test_y)

Teste Nº: 1 	 Real: Iris-virginica 	 Predito: Iris-virginica
Teste Nº: 2 	 Real: Iris-virginica 	 Predito: Iris-virginica
Teste Nº: 3 	 Real: Iris-setosa 	 Predito: Iris-setosa
Teste Nº: 4 	 Real: Iris-versicolor 	 Predito: Iris-versicolor
Teste Nº: 5 	 Real: Iris-setosa 	 Predito: Iris-setosa
Teste Nº: 6 	 Real: Iris-versicolor 	 Predito: Iris-versicolor
Teste Nº: 7 	 Real: Iris-virginica 	 Predito: Iris-versicolor
Teste Nº: 8 	 Real: Iris-versicolor 	 Predito: Iris-versicolor
Teste Nº: 9 	 Real: Iris-setosa 	 Predito: Iris-setosa
Teste Nº: 10 	 Real: Iris-setosa 	 Predito: Iris-setosa
Teste Nº: 11 	 Real: Iris-setosa 	 Predito: Iris-setosa
Teste Nº: 12 	 Real: Iris-virginica 	 Predito: Iris-virginica
Teste Nº: 13 	 Real: Iris-virginica 	 Predito: Iris-virginica
Teste Nº: 14 	 Real: Iris-versicolor 	 Predito: Iris-versicolor
Teste Nº: 15 	 Real: Iris-versicolor 	 Predito: Iris-versicolor
Teste Nº: 16 	 Real: Iris-versicolor 	 Predito: Iris-versicolor
Teste Nº: 17 	 Real: Iris-virginic

In [1393]:
# Medindo a acurácia do modelo utilizando as mesmas listas de treino e teste do caso de teste 1
KNN.accuracy_report(prediction_result, test_y)

Quantidade de Treinos: 120
Quantidade de Testes: 30
Quantidade de Acertos: 28
Quantidade de Erros: 2
Acurácia do Modelo: 93.33%


In [1394]:
# Gerando a matriz de confusão
KNN.confusion_matrix(prediction_result, test_y)

		Valores Preditos
-----------------------------------
		| [9, 0, 0] 

Valores Reais	| [0, 9, 0] 

		| [0, 2, 10] 



#### Caso de Teste 2: Utilizando o método de manhattan para cálculo da distância e k par (10):
##### Utilizando as mesmas listas de treino e teste do caso de teste 1.

In [1395]:
# Instanciando o modelo KNN a partir do código criado
KNN_2 = KNeighborsClassifier(k=10, distance_calculation_method='manhattan')

# Treinando o modelo
KNN_2.fit(train_x, train_y)

In [1396]:
# Realizando as predições com base no modelo treinado
prediction_result_2 = KNN_2.predict(test_x)

In [1397]:
# Visualizando os testes realizados
KNN_2.prediction_report(prediction_result_2, test_y)

Teste Nº: 1 	 Real: Iris-virginica 	 Predito: Iris-virginica
Teste Nº: 2 	 Real: Iris-virginica 	 Predito: Iris-virginica
Teste Nº: 3 	 Real: Iris-setosa 	 Predito: Iris-setosa
Teste Nº: 4 	 Real: Iris-versicolor 	 Predito: Iris-versicolor
Teste Nº: 5 	 Real: Iris-setosa 	 Predito: Iris-setosa
Teste Nº: 6 	 Real: Iris-versicolor 	 Predito: Iris-versicolor
Teste Nº: 7 	 Real: Iris-virginica 	 Predito: Iris-virginica
Teste Nº: 8 	 Real: Iris-versicolor 	 Predito: Iris-versicolor
Teste Nº: 9 	 Real: Iris-setosa 	 Predito: Iris-setosa
Teste Nº: 10 	 Real: Iris-setosa 	 Predito: Iris-setosa
Teste Nº: 11 	 Real: Iris-setosa 	 Predito: Iris-setosa
Teste Nº: 12 	 Real: Iris-virginica 	 Predito: Iris-virginica
Teste Nº: 13 	 Real: Iris-virginica 	 Predito: Iris-virginica
Teste Nº: 14 	 Real: Iris-versicolor 	 Predito: Iris-versicolor
Teste Nº: 15 	 Real: Iris-versicolor 	 Predito: Iris-versicolor
Teste Nº: 16 	 Real: Iris-versicolor 	 Predito: Iris-versicolor
Teste Nº: 17 	 Real: Iris-virginica

In [1398]:
# Medindo a acurácia do modelo utilizando as mesmas listas de treino e teste do caso de teste 1
KNN_2.accuracy_report(prediction_result_2, test_y)

Quantidade de Treinos: 120
Quantidade de Testes: 30
Quantidade de Acertos: 29
Quantidade de Erros: 1
Acurácia do Modelo: 96.67%


In [1399]:
# Gerando a matriz de confusão
KNN_2.confusion_matrix(prediction_result_2, test_y)

		Valores Preditos
-----------------------------------
		| [9, 0, 0] 

Valores Reais	| [0, 9, 0] 

		| [0, 1, 11] 



### Comparando o algorítmo criado pelo algorítmo no sklearn
##### Utilizando as mesmas listas de treino e teste instanciadas no caso de teste 1. 

#### Importe das bibliotecas necessárias:

In [1400]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import sklearn.neighbors

#### Comparando o caso de teste 1:

In [1401]:
# Instanciando o modelo KNN com o sklearn
KNN_sklearn = sklearn.neighbors.KNeighborsClassifier(n_neighbors = 5)

# Treinando o modelo
KNN_sklearn.fit(train_x, train_y)

# Medindo a acurácia do modelo utilizando as mesmas listas de treino e teste do caso de teste 1
KNN_sklearn.score(test_x, test_y)

0.9666666666666667

In [1402]:
# Gerando a matriz de confusão
confusion_matrix(test_y, KNN_sklearn.predict(test_x))

array([[ 9,  0,  0],
       [ 0,  9,  0],
       [ 0,  1, 11]])

#### Comparando o caso de teste 2:

In [1403]:
# Instanciando o modelo KNN com o sklearn
KNN_sklearn_2 = sklearn.neighbors.KNeighborsClassifier(n_neighbors = 10)

# Treinando o modelo
KNN_sklearn_2.fit(train_x, train_y)

# Medindo a acurácia do modelo utilizando as mesmas listas de treino e teste do caso de teste 1
KNN_sklearn_2.score(test_x, test_y)

0.9333333333333333

In [1404]:
# Gerando a matriz de confusão
confusion_matrix(test_y, KNN_sklearn_2.predict(test_x))

array([[ 9,  0,  0],
       [ 0,  9,  0],
       [ 0,  2, 10]])